<a href="https://colab.research.google.com/github/Xujjjjun2002/AI/blob/main/%E8%87%AA%E5%AE%9A%E4%B9%89%E6%95%B0%E6%8D%AE%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os,glob
import random,csv
from torch.utils.data import Dataset

Resize

Data Argumentation
(Rotate Crop)

Normalize

ToTensor

In [ ]:
class Cells_Data(Dataset):
  def __init__(self,root,resize,mode):
    super(Cells_Data,self).__init__()

    self.root = root
    self.resize = resize

    self.name2label = {}
    for name in sorted(os.listdir(os.path.join(root))):
      if not os.path.isdir(os.path.join(root,name)):
        continue

      self.name2label[name] = len(self.name2label.keys())

    print(self.name2label)

    #对象->(image,label)

    self.images,self.labels = self.load_csv('images.csv')

    if mode == 'train':
      self.images = self.images[:int(0.6*len(self.images))]
      self.labels = self.labels[:int(0.6*len(self.labels))]
    elif mode == 'val':
      self.images = self.images[int(0.6*len(self.images)):int(0.8*len(self.images))]
      self.labels = self.labels[int(0.6*len(self.labels)):int(0.8*len(self.labels))]
    else:
      self.images = self.images[int(0.8*len(self.images)):]
      self.labels = self.labels[int(0.8*len(self.labels)):]

  def load_csv(self,filename):
    if not os.path.exist(os.path.join(self.root,filename)):
      images = []
      for name in self.name2label.keys():
        images += glob.glob(os.path.join(self.root,name,'*.png'))
        images += glob.glob(os.path.join(self.root,name,'*.jpg'))
        images += glob.glob(os.path.join(self.root,name,'*.jpeg'))

      #glob函数的用法：返回路径下所有文件格式为png(jpg)的文件的路径，添加到images中去

      print("len(images)=",len(images))

      random.shuffle(images)
      with open(os.path.join(self.root,filename),mode='w',newline='') as f:
        writer = csv.writer(f)
        for img in images:
          name = img.split(os.sep)[-2]
          label = self.name2label[name]
          writer.writerow([img,label])
        print('write into csv file')

    images,labels = [],[]
    with open(os.path.join(self.root,filename)) as f:
      reader = csv.reader(f)
      for row in reader:
        img,label = row
        label = int(label)

        images.append(img)
        labels.append(label)

    assert len(images) == len(labels)

    return images,labels

  def __len__(self):
    return len(self.images)

  def __getitem(self,idx):
    #idx-[0-len(images)]
    #self.images,self.labels
    img,label = self.images[idx],self.labels[idx]

    tf = transforms.Compose([
        lambda x:Image.open(x).convert('RGB'),
        transforms.Resize((self.resize,self.resize)),
        transforms.ToTensor()
    ])

    img = tf(img)
    label = torch.tensor(label)

    return img,label


In [ ]:
def main():
  import visdom
  import time

  viz = visdom.Visdom()
  db = Cells_Data('/home/xujun/desk_top/task1/data',299,'train')
  x,y = next(iter(db))
  print('sample:',x.shape,y.shape)

In [ ]:
if __name__ == '__main__':
  main()